# 读取和处理CT文件

In [83]:
# 读取上一个notebook的数据

%run '1_candidate_&_annotation.ipynb'

In [84]:
import collections

import SimpleITK as sitk

In [85]:
series_uid = '1.3.6.1.4.1.14519.5.2.1.6279.6001.100684836163890911914061745866'

In [86]:
mhd_list = list(chain.from_iterable(glob.glob(path + f'subset{i}/{series_uid}.mhd') for i in range(2)))[0]

mhd_list

'/home/ben/Datasets/luna16/subset1/1.3.6.1.4.1.14519.5.2.1.6279.6001.100684836163890911914061745866.mhd'

原代码:

```python
mhd_list = glob.glob(path + f'subset{range(2)}/{series_uid}.mhd')
mhd_list
```

**Zane**:

This code is unable to read the contents of `subset1` after execution. How should I modify it?

**Copilot**:

你的代码试图在一个字符串中使用 `range(2)`，这会导致问题，因为 `range(2)` 实际上是一个迭代器，它生成一个序列 `[0, 1]`，而不是一个单独的数字。你需要在循环中为每个子目录生成一个文件路径。

你可以使用列表推导式和 `itertools.chain.from_iterable` 来修改你的代码，如下所示：

```python
from itertools import chain

mhd_list = list(chain.from_iterable(glob.glob(path + f'subset{i}/{series_uid}.mhd') for i in range(2)))
```

这段代码会为每个子目录生成一个文件路径，然后使用 `glob.glob` 来获取每个路径下的 .mhd 文件。最后，`chain.from_iterable` 将所有文件连接成一个列表。

In [87]:
ct_mhd = sitk.ReadImage(mhd_list)
ct_a = np.array(sitk.GetArrayFromImage(ct_mhd), dtype=np.float32)

ct_a

array([[[ -990., -1007., -1016., ...,  -988.,  -973.,  -966.],
        [ -985., -1001., -1014., ...,  -978.,  -960.,  -961.],
        [ -994., -1013., -1017., ...,  -972.,  -956.,  -969.],
        ...,
        [ -941.,  -953.,  -959., ...,  -918.,  -917.,  -943.],
        [ -960.,  -981.,  -979., ...,  -929.,  -903.,  -926.],
        [ -978.,  -980.,  -965., ...,  -953.,  -925.,  -924.]],

       [[ -996., -1019., -1019., ...,  -998.,  -997.,  -994.],
        [ -989., -1005., -1011., ...,  -991.,  -991.,  -988.],
        [ -987.,  -996.,  -996., ...,  -989.,  -993.,  -991.],
        ...,
        [ -974.,  -972.,  -953., ...,  -946.,  -949.,  -964.],
        [ -974.,  -984.,  -965., ...,  -948.,  -944.,  -963.],
        [ -989.,  -992.,  -969., ...,  -948.,  -949.,  -971.]],

       [[-1009., -1014., -1008., ..., -1010., -1014., -1011.],
        [ -997.,  -992.,  -999., ..., -1003., -1005.,  -991.],
        [ -999.,  -987.,  -981., ...,  -988.,  -994.,  -986.],
        ...,
        [ -9

In [88]:
ct_a.shape

(171, 512, 512)

> 不知道如何实现调用不同目录下的 `util`.

---

下面是对 `util` 中的函数的调用

In [89]:
IrcTuple = collections.namedtuple('IrcTuple', ['index', 'row', 'col'])
XyzTuple = collections.namedtuple('XyzTuple', ['x', 'y', 'z'])

In [90]:
def xyz2irc(coord_xyz, origin_xyz, vxSize_xyz, direction_a):
    origin_a = np.array(origin_xyz)
    vxSize_a = np.array(vxSize_xyz)
    coord_a = np.array(coord_xyz)
    cri_a = ((coord_a - origin_a) @ np.linalg.inv(direction_a)) / vxSize_a
    cri_a = np.round(cri_a)
    return IrcTuple(int(cri_a[2]), int(cri_a[1]), int(cri_a[0]))

In [91]:
def getRawCandidate(center_xyz, width_irc):
    center_irc = xyz2irc(
        center_xyz,
        origin_xyz,
        vxSize_xyz,
        direction_a,
    )

    slice_list = []
    for axis, center_val in enumerate(center_irc):
        start_ndx = int(round(center_val - width_irc[axis]/2))
        end_ndx = int(start_ndx + width_irc[axis])

        assert center_val >= 0 and center_val < hu_a.shape[axis], repr([series_uid, center_xyz, origin_xyz, vxSize_xyz, center_irc, axis])

        if start_ndx < 0:
            # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
            #     series_uid, center_xyz, center_irc, hu_a.shape, width_irc))
            start_ndx = 0
            end_ndx = int(width_irc[axis])

        if end_ndx > hu_a.shape[axis]:
            # log.warning("Crop outside of CT array: {} {}, center:{} shape:{} width:{}".format(
            #     series_uid, center_xyz, center_irc, hu_a.shape, width_irc))
            end_ndx = hu_a.shape[axis]
            start_ndx = int(hu_a.shape[axis] - width_irc[axis])

        slice_list.append(slice(start_ndx, end_ndx))

    ct_chunk = hu_a[tuple(slice_list)]

    return ct_chunk, center_irc

---

In [92]:
hu_a = ct_a

origin_xyz = XyzTuple(*ct_mhd.GetOrigin())
vxSize_xyz = XyzTuple(*ct_mhd.GetSpacing())
direction_a = np.array(ct_mhd.GetDirection()).reshape(3, 3)

In [93]:
origin_xyz, vxSize_xyz, direction_a

(XyzTuple(x=-127.755859375, y=-277.755859375, z=-351.5),
 XyzTuple(x=0.48828125, y=0.48828125, z=1.7999999523162842),
 array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]))

In [94]:
a = getRawCandidate([-30, -30, -60], [10, 10, 10])

In [95]:
type(a[0]), a[0].shape

(numpy.ndarray, (10, 10, 10))

In [96]:
type(a[1]), a[1]

(__main__.IrcTuple, IrcTuple(index=162, row=507, col=200))